In [1]:
import pandas as pd
import numpy as np
from gensim.models.word2vec import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import scale
from sklearn.svm import LinearSVC
from sklearn.metrics import roc_auc_score

In [2]:
data = pd.read_csv('/Users/matthewcassi/Documents/nhl_sentiment_working/sentiment_data/none/lemm_training.csv')
testing = pd.read_csv('/Users/matthewcassi/Documents/nhl_sentiment_working/sentiment_data/none/lemm_testing.csv')

In [3]:
data.drop('Unnamed: 0', 1, inplace=True)
testing.drop('Unnamed: 0', 1, inplace=True)

In [4]:
data = data.dropna()
testing = testing.dropna()

In [5]:
data_y = data['sentiment']
data_x = data['lemm_text']

x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.05, random_state=232)

In [6]:
n_dim = 300
#Initialize model and build vocab
w2v = Word2Vec(size=n_dim, min_count=10)
w2v.build_vocab(x_train)

In [7]:
w2v.train(x_train, total_examples=w2v.corpus_count, epochs=w2v.iter)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(49599158, 285825085)

In [8]:
#Build word vector for training set by using the average value of all word vectors in the tweet, then scale
def buildWordVector(text, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in text:
        try:
            vec += w2v[word].reshape((1, size))
            count += 1.
        except KeyError:
            continue
    if count != 0:
        vec /= count
    return vec

In [9]:
train_vecs = np.concatenate([buildWordVector(z, n_dim) for z in x_train])
train_vecs = scale(train_vecs)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [10]:
w2v.train(x_test, total_examples=w2v.corpus_count, epochs=w2v.iter)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


(2617115, 15066655)

In [11]:
test_vecs = np.concatenate([buildWordVector(z, n_dim) for z in x_test])
test_vecs = scale(test_vecs)

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [12]:
svc_c = np.linspace(0.001, 30, 20)
param_grid = {'loss': ['hinge','squared_hinge'],
              'C': svc_c}

svc = LinearSVC()
svc_cv = RandomizedSearchCV(svc, param_grid, n_iter=5, cv=3)
svc_cv.fit(train_vecs, y_train)
print(svc_cv)

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
          fit_params=None, iid=True, n_iter=5, n_jobs=1,
          param_distributions={'loss': ['hinge', 'squared_hinge'], 'C': array([  1.00000e-03,   1.57989e+00,   3.15879e+00,   4.73768e+00,
         6.31658e+00,   7.89547e+00,   9.47437e+00,   1.10533e+01,
         1.26322e+01,   1.42111e+01,   1.57899e+01,   1.73688e+01,
         1.89477e+01,   2.05266e+01,   2.21055e+01,   2.36844e+01,
         2.52633e+01,   2.68422e+01,   2.84211e+01,   3.00000e+01])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)


In [13]:
pred = svc_cv.predict(test_vecs)

In [14]:
print(svc_cv.score(test_vecs, y_test))

0.502339387231


In [15]:
print(classification_report(y_test, pred,
     target_names=['neg','pos']))

             precision    recall  f1-score   support

        neg       0.50      0.51      0.51     39579
        pos       0.50      0.49      0.50     39929

avg / total       0.50      0.50      0.50     79508



In [16]:
print(confusion_matrix(y_test, pred))

[[20270 19309]
 [20259 19670]]


In [17]:
print(roc_auc_score(y_test, pred))

0.502382342367


In [18]:
from sklearn.externals import joblib
joblib.dump(svc_cv, 'word2vec_svc_lemm.pkl') 

['word2vec_svc_lemm.pkl']